## Fake News or Not?
#### Ford St. John and Matthew Duc Nguyen
<img src="https://static01.nyt.com/images/2019/10/02/business/02TRUMPMEDIA-01/02TRUMPMEDIA-01-superJumbo.jpg" height=500 width=750>

Using the Kaggle dataset ![Fake News](https://www.kaggle.com/c/fake-news/data) we will attempt to train a machine learning model that can identify news articles as "reliable" or "unreliable" (e.g. fake) based on the following article attributes
1. id: a unique ID to identify the article
2. title: the title of the news article
3. author: the author of the news article
4. text: the content (actual text) of the news article; may or may not be incomplete

Articles are labeled with a boolean 1|0 indicator with 1 = "unreliable" and 0 = "reliable"

## Step 1: Exploratory data analysis
Before undertaking any model training, we want to get a feel for the data and identify features that we think might merit further exploration.  Our dataset has been pre-split for us into a training and testing set.  We'll use the training dataset to train our model, and use the testing dataset to test our model on data it's never seen before

In [1]:
# Package imports
import pandas as pd
import kaggle